In [ ]:
from torch.utils.data import DataLoader
from src.dataset import EASTDataset

train_dataset = EASTDataset(
    img_dir="data/icdar2015/train_images",
    map_dir="data/icdar2015/train_maps"
)

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)

imgs, scores, geos = next(iter(train_loader))

print("Images:", imgs.shape)
print("Scores:", scores.shape)
print("Geos:", geos.shape)